In [7]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import random
import math
from sklearn.metrics import *

# Setting random seeds to keep everything deterministic.
random.seed(1618)
np.random.seed(1618)
#tf.set_random_seed(1618)   # Uncomment for TF1.
tf.random.set_seed(1618)

# Disable some troublesome logging.
#tf.logging.set_verbosity(tf.logging.ERROR)   # Uncomment for TF1.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Information on dataset.
NUM_CLASSES = 10
IMAGE_SIZE = 784
NEURONS_PER_LAYER = 32
HAVE_MINI_BATCHES = False
NUM_EPOCHS = 10
MBS = 50
LR = 0.5

# Use these to set the algorithm to use.
#ALGORITHM = "guesser"
ALGORITHM = "custom_net"
#ALGORITHM = "tf_net"

class NeuralNetwork_2Layer():
    def __init__(self, inputSize, outputSize, neuronsPerLayer, learningRate = 0.1):
        self.inputSize = inputSize
        self.outputSize = outputSize
        self.neuronsPerLayer = neuronsPerLayer
        self.lr = learningRate
        self.W1 = np.random.randn(self.inputSize, self.neuronsPerLayer)
        self.W2 = np.random.randn(self.neuronsPerLayer, self.outputSize)
    def mse(self, x, y):
        return 0.5 * (x-y)**2

    # Activation function.
    def __sigmoid(self, x):
        return 1/(1+np.exp(-x))

    # Activation prime function.
    def __sigmoidDerivative(self, x):
        return x*(1-x)

    # Batch generator for mini-batches. Not randomized.
    def __batchGenerator(self, l, n):
        for i in range(0, len(l), n):
            yield l[i : i + n]

    # Training with backpropagation.
    def train(self, xVals, yVals, epochs = 10, minibatches = True, mbs = 10):
        #TODO: Implement backprop. allow minibatches. mbs should specify the size of each minibatch.
        for i in range(epochs):
            if not minibatches:
                xBatches = xVals
                yBatches = yVals
            xBatches = self.__batchGenerator(xVals, mbs)
            yBatches = self.__batchGenerator(yVals, mbs)
            for xBatch, yBatch in zip(xBatches,yBatches):
                layer1, layer2 = self.__forward(xBatch)
                y_pred = layer2

                O_error = (y_pred - yBatch)/len(yBatch)
                O_delta = O_error * self.__sigmoidDerivative(y_pred)
                self.W2 -= self.lr * layer1.T.dot(O_delta)
                
                H_error = O_delta.dot(self.W2.T)
                H_delta = H_error * self.__sigmoidDerivative(layer1)
                self.W1 -= self.lr * xBatch.T.dot(H_delta)
                
    # Forward pass.
    def __forward(self, input):
        layer1 = self.__sigmoid(np.dot(input, self.W1))
        layer2 = self.__sigmoid(np.dot(layer1, self.W2))
        return layer1, layer2

    # Predict.
    def predict(self, xVals):
        _, layer2 = self.__forward(xVals)
        return layer2

# Classifier that just guesses the class label.
def guesserClassifier(xTest):
    ans = []
    for entry in xTest:
        pred = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        pred[random.randint(0, 9)] = 1
        ans.append(pred)
    return np.array(ans)



#=========================<Pipeline Functions>==================================

def getRawData():
    mnist = tf.keras.datasets.mnist
    (xTrain, yTrain), (xTest, yTest) = mnist.load_data()
    print("Shape of xTrain dataset: %s." % str(xTrain.shape))
    print("Shape of yTrain dataset: %s." % str(yTrain.shape))
    print("Shape of xTest dataset: %s." % str(xTest.shape))
    print("Shape of yTest dataset: %s." % str(yTest.shape))
    return ((xTrain, yTrain), (xTest, yTest))



def preprocessData(raw):
    ((xTrain, yTrain), (xTest, yTest)) = raw            #TODO: Add range reduction here (0-255 ==> 0.0-1.0).
    xTrain = xTrain / 255
    xTest = xTest / 255
    if ALGORITHM == "custom_net":
        xTrain = xTrain.reshape(len(xTrain), IMAGE_SIZE)
        xTest = xTest.reshape(len(xTest), IMAGE_SIZE)
    elif ALGORITHM == "tf_net":
        xTrain = xTrain.reshape(len(xTrain), 28, 28, 1)
        xTest = xTest.reshape(len(xTest), 28, 28, 1)
    yTrainP = to_categorical(yTrain, NUM_CLASSES)
    yTestP = to_categorical(yTest, NUM_CLASSES)
    
    print("New shape of xTrain dataset: %s." % str(xTrain.shape))
    print("New shape of xTest dataset: %s." % str(xTest.shape))
    print("New shape of yTrain dataset: %s." % str(yTrainP.shape))
    print("New shape of yTest dataset: %s." % str(yTestP.shape))
    return ((xTrain, yTrainP), (xTest, yTestP))

def trainModel(data):
    xTrain, yTrain = data
    if ALGORITHM == "guesser":
        return None   # Guesser has no model, as it is just guessing.
    elif ALGORITHM == "custom_net":
        print("Building and training Custom_NN.")
        nn = NeuralNetwork_2Layer(IMAGE_SIZE, NUM_CLASSES, NEURONS_PER_LAYER, learningRate = LR)
        nn.train(xTrain, yTrain, epochs = NUM_EPOCHS, minibatches =  HAVE_MINI_BATCHES, mbs = MBS)
        return nn
    elif ALGORITHM == "tf_net":
        print("Building and training TF_NN.")
        model = keras.Sequential()
        loss = keras.losses.categorical_crossentropy
        opt = tf.compat.v1.train.AdamOptimizer()
        model.add(keras.layers.Conv2D(NEURONS_PER_LAYER, kernel_size=3, activation='relu', input_shape=(28,28,1)))
        model.add(keras.layers.Conv2D(NEURONS_PER_LAYER, kernel_size=3, activation='relu'))
        model.add(keras.layers.Flatten())
        model.add(keras.layers.Dense(NUM_CLASSES, activation='softmax'))
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        if HAVE_MINI_BATCHES:
            model.fit(xTrain, yTrain, epochs=NUM_EPOCHS, batch_size = MBS)
        else:
            model.fit(xTrain, yTrain, epochs=NUM_EPOCHS)
        return model
    else:
        raise ValueError("Algorithm not recognized.")

def runModel(data, model):
    if ALGORITHM == "guesser":
        return guesserClassifier(data)
    elif ALGORITHM == "custom_net":
        print("Testing Custom_NN.")
        return model.predict(data)
    elif ALGORITHM == "tf_net":
        print("Testing TF_NN.")
        return model.predict(data)
    else:
        raise ValueError("Algorithm not recognized.")

def evalResults(data, preds):   #TODO: Add F1 score confusion matrix here.
    xTest, yTest = data
    acc = 0
    y_actual = []
    y_pred = []
    for i in range(preds.shape[0]):
        p = np.argmax(preds[i])
        y_pred.append(p)
        y = np.argmax(yTest[i])
        y_actual.append(y)
        if p == y:
            acc = acc + 1
        accuracy = acc / preds.shape[0]
    conf_mat = confusion_matrix(y_actual, y_pred, labels=range(0,10))
    score = f1_score(y_actual, y_pred, average = "weighted")
    print("Classifier algorithm: %s" % ALGORITHM)
    print("Classifier accuracy: %f%%" % (accuracy * 100))
    print("Confusion Matrix: ")
    print(conf_mat)
    print("F1 Score: ")
    print(score)



#=========================<Main>================================================

def main():
    raw = getRawData()
    data = preprocessData(raw)
    model = trainModel(data[0])
    preds = runModel(data[1][0], model)
    evalResults(data[1], preds)

if __name__ == '__main__':
    main()

Shape of xTrain dataset: (60000, 28, 28).
Shape of yTrain dataset: (60000,).
Shape of xTest dataset: (10000, 28, 28).
Shape of yTest dataset: (10000,).
New shape of xTrain dataset: (60000, 784).
New shape of xTest dataset: (10000, 784).
New shape of yTrain dataset: (60000, 10).
New shape of yTest dataset: (10000, 10).
Building and training Custom_NN.
Testing Custom_NN.
Classifier algorithm: custom_net
Classifier accuracy: 89.400000%
Confusion Matrix: 
[[ 944    1    1    2    0   10   12    3    7    0]
 [   0 1101    4    6    1    1    4    0   17    1]
 [  19    6  883   22   11    5   16   23   35   12]
 [   3    1   25  888    1   37    7   15   26    7]
 [   2    0    6    0  899    4   11    0   11   49]
 [  18    4    9   46   26  723   18   11   29    8]
 [  21    4    8    1   15   15  890    1    3    0]
 [   5   16   27    7   10    3    0  923    7   30]
 [  14    6   14   29   13   35   23   17  814    9]
 [   9    7    5   13   46   18    3   24    9  875]]
F1 Score: 
0.